In [2]:

# To model and solve a power generation optimization problem using Gurobi in Python, you'll need to formulate your 
# problem as a mathematical optimization model. Here's a simplified example using linear programming. In a real-world 
# scenario, you might need a more sophisticated model depending on the specifics of your system.

# Let's assume you have data for water flow, weather, temperature, sunshine, rainfall over the past year, and you 
# want to maximize power generation based on these factors.

# Essentialls, gurobipy is the Python interface for Gurobi Optimization, a powerful optimization solver. 
# Gurobi Optimization is a commercial optimization solver that provides state-of-the-art performance for a wide 
# range of mathematical programming problems, including linear programming (LP), quadratic programming (QP), 
# mixed-integer programming (MIP), and more


In [3]:

import pandas as pd

# Sample data for the past year
data = {
    'Month': ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
    'Water_Flow': [100, 120, 90, 80, 110, 130, 150, 140, 120, 100, 80, 90],
    'Temperature': [15, 18, 20, 25, 28, 30, 32, 28, 25, 20, 18, 16],
    'Sunshine': [6, 7, 8, 9, 10, 11, 12, 11, 10, 8, 7, 6],
    'Rainfall': [50, 40, 30, 20, 10, 5, 5, 10, 15, 25, 35, 45],
}

df = pd.DataFrame(data)
df


import gurobipy as gp
from gurobipy import GRB

# Create a Gurobi model
model = gp.Model("power_generation")


# Variables
months = df['Month'].tolist()
water_flow = df['Water_Flow'].tolist()
temperature = df['Temperature'].tolist()
sunshine = df['Sunshine'].tolist()
rainfall = df['Rainfall'].tolist()



# Defines decision variables for power generation in each month and sets the objective to maximize 
# the total power generation over the year.
# Power generation variables for each month
power_gen = model.addVars(months, name="Power_Generation", lb=0)

# Objective: Maximize total power generation over the year
model.setObjective(gp.quicksum(power_gen[month] for month in months), sense=GRB.MAXIMIZE)


# Adds constraints for each month based on the factors: 'Water_Flow', 'Temperature', 'Sunshine', and 
# 'Rainfall'. Adjusts coefficients according to the influence of each factor on power generation.
# Constraints: Power generation based on water flow and other factors
for month in months:
    model.addConstr(power_gen[month] <= 2 * water_flow[months.index(month)], f"Water_Flow_Constraint_{month}")
    model.addConstr(power_gen[month] <= 0.5 * temperature[months.index(month)], f"Temperature_Constraint_{month}")
    model.addConstr(power_gen[month] <= 0.2 * sunshine[months.index(month)], f"Sunshine_Constraint_{month}")
    model.addConstr(power_gen[month] <= 1.5 * rainfall[months.index(month)], f"Rainfall_Constraint_{month}")

    
# Optimize the model
model.optimize()


Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 13th Gen Intel(R) Core(TM) i5-1335U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 48 rows, 12 columns and 48 nonzeros
Model fingerprint: 0x0eb96b49
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+02]
Presolve removed 48 rows and 12 columns
Presolve time: 0.04s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.05 seconds (0.00 work units)
Optimal objective  2.100000000e+01


In [4]:

# Print the optimal solution
print("Optimal Power Generation:")
for month in months:
    print(f"{month}: {power_gen[month].x} MW")
    

Optimal Power Generation:
Jan: 1.2000000000000002 MW
Feb: 1.4000000000000001 MW
Mar: 1.6 MW
Apr: 1.8 MW
May: 2.0 MW
Jun: 2.2 MW
Jul: 2.4000000000000004 MW
Aug: 2.2 MW
Sep: 2.0 MW
Oct: 1.6 MW
Nov: 1.4000000000000001 MW
Dec: 1.2000000000000002 MW


In [1]:

# This code sets up an optimization model using Gurobi to maximize power generation over the year while 
# considering constraints based on water flow, temperature, sunshine, and rainfall. The coefficients in 
# the constraints can be adjusted to reflect the impact of each factor on power generation. The final output 
# shows the optimal power generation values for each month.
